<a href="https://colab.research.google.com/github/lro99/fraud_classification/blob/main/Fraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from google.colab import drive
random.seed(42)

In [2]:
# drive.mount('/content/drive')

In [3]:
train = pd.read_parquet('/content/drive/MyDrive/Documents/Data Career/fraud_classification/train-00000-of-00002.parquet')
train.to_csv('fraud0.csv')

train1 = pd.read_parquet('/content/drive/MyDrive/Documents/Data Career/fraud_classification/train-00001-of-00002.parquet')
train1.to_csv('fraud1.csv')

In [4]:
# train1.head()

In [5]:
# train.head()

In [6]:
train_df = pd.concat([train, train1], ignore_index=True)

# fraction = 0.55
# train_df = train_df_unsampled.sample(frac=fraction, random_state=42)

train_df.head()

,ssn,cc_num,first,last,gender,city,state,zip,city_pop,job,dob,acct_num,trans_num,trans_date,trans_time,unix_time,category,amt,is_fraud,merchant
0,367-85-9826,4361337605230458,Kristie,Davis,F,Chandler,OK,74834,7590,Chief Strategy Officer,1987-06-12,349734538563,c036244703adb9d5392f4027d9d4b38d,2021-07-31,02:30:01,1627678801,grocery_pos,337.54,1,fraud_Kovacek
1,367-85-9826,4361337605230458,Kristie,Davis,F,Chandler,OK,74834,7590,Chief Strategy Officer,1987-06-12,349734538563,42f000b0b3b0ef534e5b8ef9ec1db13a,2021-08-01,22:37:41,1627837661,personal_care,21.13,1,fraud_Bradtke
2,367-85-9826,4361337605230458,Kristie,Davis,F,Chandler,OK,74834,7590,Chief Strategy Officer,1987-06-12,349734538563,543037b1baf088961e58d00b705f4bcc,2021-08-01,23:02:09,1627839129,personal_care,22.61,1,fraud_Kozey-Kuhlman
3,367-85-9826,4361337605230458,Kristie,Davis,F,Chandler,OK,74834,7590,Chief Strategy Officer,1987-06-12,349734538563,00a4e08643edebf9277c2967676f6a26,2021-08-01,22:27:24,1627837044,health_fitness,17.32,1,fraud_Hills
4,367-85-9826,4361337605230458,Kristie,Davis,F,Chandler,OK,74834,7590,Chief Strategy Officer,1987-06-12,349734538563,492c4412815306718f686fc5b459a285,2021-12-02,02:28:51,1638392331,misc_pos,75.82,0,fraud_Kemmer-Buckridge


In [7]:
# train_df.dtypes

In [8]:
# encoding

# ssn: drop dash, convert to int
# first: drop
# last: drop
# gender: onehot encode
# city & state: concatenate
# job: label encode
# dob: drop dash, convert to int
# transnum: potentially drop
# transdate: drop
# transtime: drop
# category: label encode
# merchant: label encode

In [9]:
# conversions
train_df['ssn'] = train_df['ssn'].str.replace('-', '').astype(int)
train_df['dob'] = train_df['dob'].str.replace('-', '').astype(int)

train_df['location'] = train_df['city'] + ', ' + train_df['state']

# onehot encode (gender)
onehot = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
onehot.fit(train_df[['gender']])

feature_names = onehot.get_feature_names_out(['gender'])
gender_encoded = pd.DataFrame(onehot.transform(train_df[['gender']]), columns=feature_names)

# label encode (job, category, merchant)
encode_df = train_df[['job', 'category', 'merchant', 'location']]
encoder = LabelEncoder()
for col in encode_df.columns:
    encode_df[col] = encoder.fit_transform(encode_df[col])

# drop unneeded columns
train_df = train_df.drop(['job', 'category', 'merchant', 'first', 'last', 'trans_num', 'trans_date', 'trans_time', 'city', 'state', 'gender', 'location'], axis=1)
train_df.head()


<ipython-input-9-2f29263f5388>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  encode_df[col] = encoder.fit_transform(encode_df[col])
<ipython-input-9-2f29263f5388>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  encode_df[col] = encoder.fit_transform(encode_df[col])
<ipython-input-9-2f29263f5388>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

,ssn,cc_num,zip,city_pop,dob,acct_num,unix_time,amt,is_fraud
0,367859826,4361337605230458,74834,7590,19870612,349734538563,1627678801,337.54,1
1,367859826,4361337605230458,74834,7590,19870612,349734538563,1627837661,21.13,1
2,367859826,4361337605230458,74834,7590,19870612,349734538563,1627839129,22.61,1
3,367859826,4361337605230458,74834,7590,19870612,349734538563,1627837044,17.32,1
4,367859826,4361337605230458,74834,7590,19870612,349734538563,1638392331,75.82,0


In [10]:
final_df = pd.concat([train_df, gender_encoded, encode_df], axis=1)
final_df.head()

,ssn,cc_num,zip,city_pop,dob,acct_num,unix_time,amt,is_fraud,gender_F,gender_M,job,category,merchant,location
0,367859826,4361337605230458,74834,7590,19870612,349734538563,1627678801,337.54,1,1.0,0.0,72,4,317,116
1,367859826,4361337605230458,74834,7590,19870612,349734538563,1627837661,21.13,1,1.0,0.0,72,10,67,116
2,367859826,4361337605230458,74834,7590,19870612,349734538563,1627839129,22.61,1,1.0,0.0,72,10,320,116
3,367859826,4361337605230458,74834,7590,19870612,349734538563,1627837044,17.32,1,1.0,0.0,72,5,238,116
4,367859826,4361337605230458,74834,7590,19870612,349734538563,1638392331,75.82,0,1.0,0.0,72,9,283,116


In [11]:
# final_df.dtypes

y = final_df['is_fraud']
X = final_df.drop(['is_fraud'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(2117355, 14) (2117355,)
(529339, 14) (529339,)


In [12]:
# VIF to determine multicollinearity

from statsmodels.stats.outliers_influence import variance_inflation_factor

# Assuming X is your feature matrix (excluding the target variable)
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)


      Feature       VIF
0         ssn  1.004902
1      cc_num  1.007605
2         zip  1.019979
3    city_pop  1.021233
4         dob  0.223602
5    acct_num  1.010276
6   unix_time  0.778788
7         amt  1.000748
8    gender_F  1.012980
9    gender_M  1.014168
10        job  1.012588
11   category  1.066931
12   merchant  0.999868
13   location  1.014297


In [13]:
import statsmodels.api as sm
# Add a constant to the independent variables
X_train_sm = sm.add_constant(X_train)

# Fit the logistic regression model
model = sm.Logit(y_train, X_train_sm).fit()
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.021788
         Iterations 11
                           Logit Regression Results                           
Dep. Variable:               is_fraud   No. Observations:              2117355
Model:                          Logit   Df Residuals:                  2117341
Method:                           MLE   Df Model:                           13
Date:                Wed, 12 Mar 2025   Pseudo R-squ.:                 0.07631
Time:                        01:20:15   Log-Likelihood:                -46134.
converged:                       True   LL-Null:                       -49945.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         26.4317        nan        nan        nan         nan         nan
ssn         1.294e-11   4.55

In [14]:
# make predictions

X_test_sm = sm.add_constant(X_test)
y_pred = model.predict(X_test_sm)
y_pred

,0
2205862,0.003005
1593920,0.003458
2143506,0.004124
325191,0.003948
2595224,0.001705
...,...
614058,0.001798
1509099,0.001884
1349835,0.004186
776959,0.003710


In [15]:
y_pred_binary = (y_pred >= 0.5).astype(int)

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred_binary)
print(cm)

[[527343     98]
 [  1898      0]]


In [16]:
# accuracy score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)

print(accuracy)
print(precision)

0.996229259510446
0.0


In [17]:
# saving the file
import pickle
file_path = '/content/drive/MyDrive/Documents/Data Career/fraud_classification/fraud_class.pkl'

with open(file_path, "wb") as f:
    pickle.dump(model, f)